In [ ]:
actions = ['a', 'b', 'c', 'd']
rules = [('ab', ''), 
         ('cd', '')
         ('aaaaa', '')
         ('bbbbb', '')
         ('ccc', '')
         ('ddd', '')
         ('aP', 'P'),
         ('accP', 'cP')]

In [2]:
initial_graph = [
    ('P', (None, [1])),
    ('cP', (None, [0]))
]

In [4]:
def duplicate_graph(graph, T):
    l = len(graph)
    old_graph = []
    new_graph = []
    for formula, (mod, neighbours) in graph:
        old_graph.append((formula, (mod, [n for n in neighbours] )) )        
        new_graph.append((T + formula, (mod, [n+l for n in neighbours] )) )
    return old_graph + new_graph

In [23]:
import copy
def clone_graph(graph):
    return [(f, (m, copy(ns))) for (f, (m, ns)) in graph]

In [7]:
def apply_prefix_rules(graph, rules):
    new_graph = []
    for formula, (mod, neighbours) in graph:
        for (prefix, replacement) in rules:
            if formula.startswith(rule):
                new_formula = replacement + formula[len(prefix)+1:]
            else:
                new_formula = formula
            new_graph.append((new_formula, (mod, neighbours)))

In [ ]:
def intersection_with_indices(l1, l2):
    common_members = set(l1).intersection(l2)
    return [(x, l1.index(x), l2.index(x)) for x in common_members]

In [46]:
def common_cycle(l1, l2):
    from itertools import repeat, chain
    if len(set(l1)) < len(l1):
        raise Exception('Non unique members in list 1')
    if len(set(l2)) < len(l2):
        raise Exception('Non unique members in list 2')
    tpls = sorted(intersection_with_indices(l1, l2), key= lambda x: x[1])
    result = []
    overboard = False
    for i, prv, tpl in zip(range(len(tpls)), tpls, tpls[1:]):
        if tpl[2] < prv[2]:
            if overboard:
                raise Exception('Ordering is not cyclic')
            overboard = True
            gap = (tpl[1] - prv[1]) - (tpl[2] + len(l2) - prv[2])
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], len(l2))] + \
                 list(repeat(None, gap)) + \
                 [l2[i] for i in range(0, tpl[2])]
            result = result + zip(s1, s2)
        elif tpl[2] - prv[2] != tpl[1] - prv[1]:
            raise Exception('Mismatch in index {} (sorted)'.format(i))
        else:
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            result = result + zip(s1, s2)           
    prv = tpls[-1]
    tpl = tpls[0]
    gap = (tpl[2] - prv[2]) - (tpl[1] + len(l1) - prv[1])
    s2 = [l2[i] for i in range(prv[2], tpl[2])]
    s1 = [l1[i] for i in range(prv[1], len(l1))] + \
         list(repeat(None, gap)) + \
         [l1[i] for i in range(0, tpl[1])]
    result = result + zip(s1, s2)
    return result
                

In [49]:
common_cycle([1,2,3,4], [5,3,9,1])

[(1, 1), (2, 5), (3, 3), (4, 9)]

In [12]:
def common_member(l1, l2):    
    for i1 in range(len(l1)):
        x1 = l1[i1]
        if x1 in l2:
            return (i1, l2.index(l1[i1]))
    return (-1, -1)

In [15]:
def index_or_identity(l, i):
    return l[i] if i in l else i

In [21]:
def without(l, indices):
    return [l[i] for i in range(len(l)) if i not in indices]

In [24]:
from functools import partial
def merge_vertices(graph, i, j, merge_table={}):
    '''
    vertex of index i is merged into j
    '''
    if merge_table.get(i, j) != j:
        raise Exception('Merge conflict: {} cannot be merged into {} it is already planned to be merged into {}'.format(
                i, j, merge_table[i]))
    f_i, (mod_i, ns_i) = graph[i]
    f_j, (mod_j, ns_j) = graph[j]
    if f_i != f_j:
        raise Exception('Merge conflict: {} cannot be merged into {}, formula mismatch: {} != {}'.format(
                i, j, f_i, f_j))
    merged_i = map(partial(index_or_identity, merge_table), ns_i),
        map(partial(index_or_identity, merge_table), ns_j))

        if mod_i == mod_j and mod_i is not None and set(ns_i) == set(ns_j):
        return without(graph, {i, j}) + [(f_i, (mod_i, ns_i))]

    new_mod = None
    if mod_i == mod_j:
        new_mod = mod_i
    elif mod_i is not None and mod_j is not None:
        raise Exception('Merge conflict between {} and {}, mod mismatch: {} != {}'.format(
                    i, j, mod_i, mod_j))
    elif mod_i is None:
        new_mod = mod_j
    else:
        new_mod = mod_i
    
    if new_mod is None:
        new_cycle = common_cycle()
    merged_ns_j = map(partial(index_or_identity, merge_table), ns_j)
    
    
    new_graph = clone_graph(graph)
    new_merge_table = dict(merge_table, i=j)
    for k in range(new_mod):
        n_i = ns_i[(c_i + k) % new_mod] 
        n_j = ns_j[(c_j + k) % new_mod]
        if n_i != n_j:
            new_graph = merge_vertices(new_graph, n_i, n_j, new_merge_table)
    return new_graph
        

SyntaxError: invalid syntax (<ipython-input-24-6789dbee5414>, line 20)

In [5]:
duplicate_graph(initial_graph, 'a')

[('P', (None, [1])),
 ('cP', (None, [0])),
 ('aP', (None, [3])),
 ('acP', (None, [2]))]

In [18]:
a = [1, 4]
b = [1, 4]
a == b

True

In [19]:
a.pop(0)

1

In [20]:
a

[4]

In [22]:
5 % 2

1

In [29]:
def reverse_index(l):
    return {l[i]: i for i in range(len(l))}

In [30]:
reverse_index([1, 2, 3])

{1: 0, 2: 1, 3: 2}

In [ ]:
def progress(lists, indices, index_index):
    new_indice = 
    return (lists, new_indices)

In [38]:
common_cycle([4,9,6,7,8,10,3], [2,3,19,4,5,6])

[(4, 0, 3), (6, 2, 5), (3, 6, 1)]
K
O


[(4, 4), (9, 5), (6, 6), (7, None), (8, None), (10, 2), (3, 3), (None, 19)]

In [39]:
common_cycle([2,3,19, 4,5,6], [4,9,6,7,8,10,3])

[(3, 1, 6), (4, 3, 0), (6, 5, 2)]
O
K


[(3, 3), (19, None), (4, 4), (5, 9), (6, 6), (None, 7), (None, 8), (2, 10)]

In [1]:
from itertools import repeat

In [3]:
list(repeat(1, 5))

[1, 1, 1, 1, 1]

In [24]:
range(2,4)

[2, 3]

In [7]:
intersection_with_indices([1,2,3], [3,1])

[(1, 0, 1), (3, 2, 0)]

In [8]:
list(enumerate(range(10), 1))

[(1, 0),
 (2, 1),
 (3, 2),
 (4, 3),
 (5, 4),
 (6, 5),
 (7, 6),
 (8, 7),
 (9, 8),
 (10, 9)]